# Recap: indexing
In the last mission, we explored how to speed up SELECT queries that only filter on one column by creating an index for that column. In this mission, we'll explore how to **create indexes for speeding up queries that filter on multiple columns**, like: 


In [15]:
import sqlite3

conn = sqlite3.connect("factbook.db")
print(conn.execute("SELECT * FROM facts WHERE population > 1000000 AND population_growth < 2.0;").fetchall()[:5])

[(2, 'al', 'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3), (3, 'ag', 'Algeria', 2381741, 2381741, 0, 39542166, 1.84, 23.67, 4.31, 0.92), (7, 'ar', 'Argentina', 2780400, 2736690, 43710, 43431886, 0.93, 16.64, 7.33, 0.0), (8, 'am', 'Armenia', 29743, 28203, 1540, 3056382, 0.15, 13.61, 9.34, 5.8), (9, 'as', 'Australia', 7741220, 7682300, 58920, 22751014, 1.07, 12.15, 7.14, 5.65)]


In [16]:

query_plan_one = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
print(query_plan_one)

[(0, 0, 0, 'SCAN TABLE facts')]


In [17]:
conn.execute("CREATE INDEX pop_idx ON facts(population);")
conn.execute("CREATE INDEX pop_growth_idx ON facts(population_growth);")

query_plan_two = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_two)

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]


SQLite struggles to take advantage of both indexes since each index is optimized for lookups on just that column. SQLite can use the indexes to quickly find the row id values where either population is greater than 1000000 or where population_growth is less than 0.05. If SQLite uses the index of population values to return all of the row id values where population is less than 1000000, it can't use those id values to search the pop_growth_idx index quickly to find the rows where population_growth is less than 0.05.

If you look at the query plan, you can infer that SQLite first decided to use the pop_growth_idx index to return the id values for the rows where population_growth was less than 0.05. Then, SQLite used a binary search on the facts table to access the row at each id value, add that row to a temporary collection if the value for population was greater than 1000000, and return the collection of rows.

You may be wondering why SQLite chose the pop_growth_idx instead of the pop_idx. This is because when there are 2 possible indexes available, SQLite tries to estimate which index will result in better performance. Unfortunately, to keep SQLite lightweight, limited ability was added to estimate and plan accurately and SQLite often ends up picking an index at random.

# Multi-column Indexing


In cases like this, we need to **create a multi-column index** that **contains values from both of the columns we're filtering on**. This way, both criteria in the WHERE statement can be evaluted in the index itself and the facts table will only be queried at the end when we have the specific row id values.

While the single column indexes we've created in the past contain just the primary key column (population) and the row id (id) columns, this multi-column index contains the population_growth column as well. SQLite can:

* use binary search to find the first row in this index where population is greater than 1000000,
* add the row to a temporary collection if population_growth is less than 0.05,
* advance to the next row (the index is ordered by population),
* add the row to a temporary collection if population_growth is less than 0.05,
* when the end of the index is reached, look up each row in facts using the id values from the temporary collection.


When creating a multi-column index, we need to specify **which of the columns we want as the primary key**. In the example above, this means that SQLite can use binary search to quickly jump to the first row that matches a specific population value but not for jumping to the first row that matches a specific population_growth value.


To create a multi-column index, we use the same CREATE INDEX syntax as before but instead specify 2 columns in the ON statement. The important thing to know here is that the **first column in the parentheses becomes the primary key for the index**.

In [18]:
conn = sqlite3.connect("factbook.db")

conn.execute("CREATE INDEX pop_pop_growth_idx ON facts(population, population_growth);")
query_plan_three = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
print(query_plan_three)

[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_pop_growth_idx (population>?)')]


## Covering index

This time, SQLite used the multi-column index pop_pop_growth_idx that we created instead of either pop_growth_idx or pop_idx. SQLite only needed to access the facts table to return the rest of the column values for the rows that met the WHERE criteria. This is only because the pop_pop_growth_idx doesn't contain the other values (besides population and population_growth already).

What if we restricted the columns in the SELECT that we want returned to just population and population_growth? In this case, SQLite will not need to interact with the facts table since the pop_pop_growth_idx can service the query. When an index contains all of the information necessary to answer a query, it's called a **covering index**. 

In [20]:
query_plan_four = conn.execute("EXPLAIN QUERY PLAN SELECT population, population_growth FROM facts WHERE population > 1000000 AND population_growth < 0.05;").fetchall()
print(query_plan_four)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]


Even though the query plan indicates that a binary search on facts was performed, this is misleading and it was instead able to use the covering index. You can read more about that on the documentation.

Covering indexes don't apply just to multi-column indexes. If a query we write only touches a column in the database that we have a single-column index for, SQLite will use only the index to service the query. Let's test this by writing a query that can take advantage of just the index, pop_idx, for the population column.

In [21]:

query_plan_five = conn.execute("EXPLAIN QUERY PLAN SELECT population FROm facts WHERE population > 1000000;").fetchall()
print(query_plan_five)

[(0, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_idx (population>?)')]
